<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/24_7POCKETOPTIONS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from ta.trend import MACD, SMAIndicator
from ta.momentum import StochasticOscillator, RSIIndicator
import matplotlib.pyplot as plt

# Define the stock symbols (or crypto/forex symbols)
stock_symbols = ['BTC-USD', 'ETH-USD', 'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCAD=X']
time_interval = '1h'  # Change to 1-hour intervals (valid)

# Function to process each stock or forex pair and predict buy/sell
def predict_for_symbol(symbol):
    # Fetch data from Yahoo Finance for the last 1 year with 1-hour intervals
    try:
        data = yf.download(symbol, period='1y', interval=time_interval)

        # Feature Engineering - Moving Averages, MACD, Stochastic Oscillator, RSI

        # 1. Moving Averages (SMA - Simple Moving Averages)
        data['SMA_10'] = SMAIndicator(data['Close'], window=10).sma_indicator()
        data['SMA_30'] = SMAIndicator(data['Close'], window=30).sma_indicator()

        # 2. MACD (Moving Average Convergence Divergence)
        macd = MACD(close=data['Close'], window_slow=26, window_fast=12, window_sign=9)
        data['MACD'] = macd.macd()
        data['MACD_signal'] = macd.macd_signal()

        # 3. Stochastic Oscillator
        stoch = StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
        data['stoch_k'] = stoch.stoch()
        data['stoch_d'] = stoch.stoch_signal()

        # 4. Relative Strength Index (RSI)
        rsi = RSIIndicator(close=data['Close'], window=14)
        data['RSI'] = rsi.rsi()

        # Drop NaN values (as some indicators are based on rolling windows)
        data.dropna(inplace=True)

        # Define buy/sell signal: Buy if MACD > Signal and SMA_10 > SMA_30, otherwise sell
        def buy_sell_signal(row):
            if row['MACD'] > row['MACD_signal'] and row['SMA_10'] > row['SMA_30']:
                return 1  # Buy
            else:
                return 0  # Sell

        data['Signal'] = data.apply(buy_sell_signal, axis=1)

        if data.shape[0] > 10:  # Make sure there are enough rows

            # Define features (X) and target variable (y)
            feature_columns = ['SMA_10', 'SMA_30', 'MACD', 'MACD_signal', 'stoch_k', 'stoch_d', 'RSI']
            X = data[feature_columns]
            y = data['Signal']

            # Split the data into training and test sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

            # Use RandomForest without GridSearchCV to speed up execution
            model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
            model.fit(X_train, y_train)

            # Predict the next hour signal using the last available data point
            next_hour_data = pd.DataFrame([data.iloc[-1][feature_columns]])
            next_hour_prediction = model.predict(next_hour_data)[0]

            # Output decision based on next hour prediction
            if next_hour_prediction == 1:
                decision = "BUY"
            else:
                decision = "SELL"

            # Print the result for the symbol
            print(f"For {symbol}, the recommendation for the next hour is to {decision}.")
        else:
            print(f"Not enough data to make a prediction for {symbol}.")
    except Exception as e:
        print(f"Data for {symbol} could not be fetched. Error: {e}")

# Loop through each symbol and make the prediction
for symbol in stock_symbols:
    predict_for_symbol(symbol)


[*********************100%***********************]  1 of 1 completed


For BTC-USD, the recommendation for the next hour is to SELL.


[*********************100%***********************]  1 of 1 completed


For ETH-USD, the recommendation for the next hour is to BUY.


[*********************100%***********************]  1 of 1 completed


...

For GBPUSD=X, the recommendation for the next hour is to BUY.


[*********************100%***********************]  1 of 1 completed


For USDJPY=X, the recommendation for the next hour is to SELL.


[*********************100%***********************]  1 of 1 completed


For USDCAD=X, the recommendation for the next hour is to SELL.
